# Preparation : Load Library

Pada Bagian ini kita akan melakukan sebuah operasi **Neural Network** sederhana.
Dengan 2 Input dan 1 output dan 2 hidden layer dengan 3 node.
maka akan terdapat node sebagai berikut: 
```
[2 3 3 1]
```
dan network sebagai berikut
```
   0   0   0       
       0   0   0  
   0   0   0      

   a   b   c   d
```
dengan demikian akan ada 3 matrix network:
```
ab = [2, 3]
bc = [3, 3]
cd = [3, 1]
```

In [1]:
import tensorflow.keras.backend as k
import tensorflow as tf
import numpy as np

# Membuat Dataset

Di sini kita akan membuat set data untuk keperluan training
dan data yang kita gunakan adalah data logic XOR dengan 2 input dan 1 output.

```
0 XOR 0 -> 0
0 XOR 1 -> 1
1 XOR 0 -> 1
1 XOR 1 -> 0
```

In [2]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])

Y = np.array([[0],
              [1],
              [1],
              [0]])

# Membuat tensor

In [3]:
X = tf.convert_to_tensor(X, dtype=tf.float16)
Y = tf.convert_to_tensor(Y, dtype=tf.float16)

# Randomly initialise two weight matrics
```
w1 is a matric with dims [2, 3]
w2 is a matric with dims [3, 3]
w3 is a matric with dims [3, 1]
```
Coastin them to variable since value can be changed during optimisation

In [4]:
w1 = tf.Variable(np.random.randn(2, 3), dtype=tf.float16)
w2 = tf.Variable(np.random.randn(3, 3), dtype=tf.float16)
w3 = tf.Variable(np.random.randn(3, 1), dtype=tf.float16)

# Create a function that propagates X throught the network

In [5]:
@tf.function
def forward(X, w1, w2, w3):
    X = tf.sigmoid(tf.matmul(X, w1))
    X = tf.sigmoid(tf.matmul(X, w2))
    X = tf.sigmoid(tf.matmul(X, w3))
    return X

In [6]:
print(forward(X, w1, w2, w3))

tf.Tensor(
[[0.4077]
 [0.4268]
 [0.3862]
 [0.4016]], shape=(4, 1), dtype=float16)


From the previous sliide we can see that
he model is just guessing. we need to implement
a loss function that tells the optimizer how far off
the prediction are from the actual target values.

In [7]:
@tf.function
def loss(predicted_y, target_y):
    return tf.reduce_mean(k.binary_crossentropy(target_y, predicted_y))

Train Function's role to converge the model towards the actual target value

In [8]:
@tf.function
def train(inputs, outputs, learning_rate):
    with tf.GradientTape() as tape:
        # Calculate loss
        current_loss = loss(forward(X, w1, w2, w3), outputs)
    # find gradien between the loss and each weight matrix
    dW1, dW2, dW3 = tape.gradient(current_loss, [w1, w2, w3])
    # backpropate and tune weights
    w1.assign_sub(learning_rate * dW1)
    w2.assign_sub(learning_rate * dW2)
    w3.assign_sub(learning_rate * dW3)
    del tape

Train the model for a number of iteration (epochs) Overtime the loss outputted onto the console will decrease, this means that your model is successfully training.

In [9]:
epochs = range(10000)
learning_rate = 0.1

for epochs in epochs:
    # Calculate loss to output.
    current_loss = loss(forward(X, w1, w2, w3), Y)
    # Forward the input data and tweak weights.
    train(X, Y, learning_rate)
print("Current Loss: {:2.5f}".format(current_loss))

Current Loss: 0.11194


# Test the model with a suitable domain

In [10]:
x_ = tf.convert_to_tensor(np.array([[0, 0]]), dtype=tf.float16)
print(f"0 XOR 0 -> {forward(x_, w1, w2, w3).numpy()} = { (0, 1)[ float(forward(x_, w1, w2, w3).numpy()[0][0]) > 0.5 ]}")

x_ = tf.convert_to_tensor(np.array([[0, 1]]), dtype=tf.float16)
print(f"0 XOR 1 -> {forward(x_, w1, w2, w3).numpy()} = { (0, 1)[ float(forward(x_, w1, w2, w3).numpy()[0][0]) > 0.5 ]}")

x_ = tf.convert_to_tensor(np.array([[1, 0]]), dtype=tf.float16)
print(f"1 XOR 0 -> {forward(x_, w1, w2, w3).numpy()} = { (0, 1)[ float(forward(x_, w1, w2, w3).numpy()[0][0]) > 0.5 ]}")

x_ = tf.convert_to_tensor(np.array([[1, 1]]), dtype=tf.float16)
print(f"1 XOR 1 -> {forward(x_, w1, w2, w3).numpy()} = { (0, 1)[ float(forward(x_, w1, w2, w3).numpy()[0][0]) > 0.5 ]}")

0 XOR 0 -> [[0.1501]] = 0
0 XOR 1 -> [[0.8926]] = 1
1 XOR 0 -> [[0.8975]] = 1
1 XOR 1 -> [[0.06082]] = 0
